In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import os
print(os.listdir("../input"))


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim 
from gensim.models import Word2Vec


In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
print(train_df.shape, test_df.shape)


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # remove everything that isn't word or space
    text = re.sub(r'\_', '', text)      # remove underscore
    return text


In [ ]:
# clean train_df['text']
train_df['text'] = train_df['text'].map(lambda x: clean_text(x))
train_df['text'] = train_df['text'].map(lambda x: x.strip().split())
train_df.head()


In [ ]:
test_df['text'] = test_df['text'].map(lambda x: clean_text(x))
test_df['text'] = test_df['text'].map(lambda x: x.strip().split())
test_df.head()


In [ ]:
data = []  
# iterate through each row in train_df 
for i in range(len(train_df)):
    data.append(train_df['text'][i])
for j in range(len(test_df)):
    data.append(test_df['text'][j])


In [ ]:
print(len(data))


In [ ]:
# Create CBOW model 
embedding = gensim.models.Word2Vec(data, size = 50, window = 5, min_count = 1, workers=4)


In [ ]:
embedding.train(data,total_examples=len(data),epochs=30)


In [ ]:
# convert author labels into one-hot encodings
train_df['author'] = pd.Categorical(train_df['author'])
df_Dummies = pd.get_dummies(train_df['author'], prefix='author')
train_df = pd.concat([train_df, df_Dummies], axis=1)
# Check the conversion
train_df.head()


In [ ]:
X = train_df['text']
Y = train_df[['author_EAP', 'author_HPL', 'author_MWS']].values


In [ ]:
print(X.shape, X[0])
print(Y.shape, Y[0])


In [ ]:
X_test = test_df['text']
print(X_test.shape, X_test[0])


In [ ]:
def text_to_avg(text):
    """Given a list of words, extract the respective GloVe representations
    and average the values into a single vector encoding the text meaning."""
    # initialize the average word vector
    avg = np.zeros((50,))
    # average the word vector by looping over the words in text
    for w in text:
        avg += embedding[w]
    avg = avg/len(text)
    return avg


In [ ]:
X_avg = np.zeros((X.shape[0], 50)) # initialize X_avg
for i in range(X.shape[0]):
    X_avg[i] = text_to_avg(X[i])


In [ ]:
print(X_avg.shape)
print(X_avg[0])


In [ ]:
X_test_avg = np.zeros((X_test.shape[0], 50)) # initialize X_test_avg
for i in range(X_test.shape[0]):
    X_test_avg[i] = text_to_avg(X_test[i])


In [ ]:
print(X_test_avg.shape)
print(X_test_avg[0])


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, Y_train, Y_dev = train_test_split(X_avg, Y, test_size=0.2, random_state=123)
print(X_train.shape, Y_train.shape, X_dev.shape, Y_dev.shape)


In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(50, activation='relu', input_shape=(50,)))
model.add(layers.Dense(3, activation='softmax'))

model.summary()


In [ ]:
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# train and validate the model for 30 epochs
history = model.fit(X_train, Y_train, epochs=30, batch_size=None, validation_data=(X_dev, Y_dev))


In [ ]:
# plot and visualise the training and validation losses
loss = history.history['loss']
dev_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, dev_loss, 'b', label='validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
model = models.Sequential()
model.add(layers.Dense(50, activation='relu', input_shape=(50,)))
model.add(layers.Dense(3, activation='softmax'))
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# train the model for 10 epochs
model.fit(X_avg, Y, epochs=10, batch_size=None)


In [ ]:
# predict on test set
preds = model.predict(X_test_avg)
print(preds.shape)
print(preds[7])


In [ ]:
# set the predicted labels to be the one with the highest probability
pred_labels = []
for i in range(len(X_test_avg)):
    pred_label = np.argmax(preds[i])
    pred_labels.append(pred_label)


In [ ]:
print(pred_labels[7])


In [ ]:
result = pd.DataFrame(preds, columns=['EAP','HPL','MWS'])
result.insert(0, 'id', test_df['id'])
result.head()


In [ ]:
# Generate submission file in csv format
result.to_csv('submission.csv', index=False, float_format='%.20f')
